# Is Sympy really 'slow'?
I don't want to be a bad carpenter who blames his tools. Here're my attempts at understanding why my Sympy code is slow. The technical specs of my computer may have a lot to do with the long time things are taking to run, though, any techniques I discover to speed up computation will have a speed up on any system I suppose. 

Ultimately my goal is to have integral and linear algebra routines that can run at sub-second speed, and preferrably a few ms. 

## Potential things to try out 

* ~~I may have been using the wrong Python version of Theano. The [docs](https://theano-pymc.readthedocs.io/en/latest/requirements.html) say official support for Theano is only there for Python <3.6.~~ 
    * ~~install Sympy, Theano nd pyMC3~~ *This doesn't work because Theano didn't seem to recognise 'external' sympy functions, eg. legendre*
    
* With ```lambdify``` use the 'mpmath' module as a backend. This *really* sped up computations, in comparison to the default (sympy?) backend module. 

* ~~Linux specific: change the 'niceness'of the process and so the CPU spends more time on the code.~~~

* 

In [44]:
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects
from gmpy2 import *
import matplotlib.pyplot as plt
import mpmath
from mpmath import mpf
dps = 30; mpmath.mp.dps = dps
import numpy as np
from scipy.special import jv as bessel_firstkind
from symengine import * 
import sympy
from sympy import jn,yn ,symbols, legendre, sin, cos, tan, summation, I, oo, diff, pi
from sympy import factor_terms, Matrix, besselj, bessely
from sympy import Abs, lambdify, integrate, expand,integrate, Integral
from sympy.printing.theanocode import theano_function
from sympy.utilities.autowrap import autowrap
import tqdm
x, alpha, k, m,n,p, r1, R, theta = symbols('x alpha k m n p r1 R theta')
from sympy import N, cse

from sympy.printing.theanocode import theano_function
num_sumterms = 50

In [2]:
from sympy import besselj

In [3]:
I*yn(10,0.5)

I*yn(10, 0.5)

In [47]:
sph_bessel1 = lambdify([n,p], besselj(n+1/2,p)*sqrt(pi/2))
sph_bessel2 = lambdify([n,p], bessely(n+1/2,p)*sqrt(pi/2))
#sph_hankel2 = lambdify([n, p], sph_bessel1(n,p)-I*sph_bessel2(n,p), 'mpmath')

In [5]:
sph_hankel2(1,0.2)

jn(1, 0.2) + 1.0*I*yn(1, 0.2)

In [6]:
r1 = (R*cos(alpha))/cos(theta)
Lm_expr = expand(legendre(m, cos(theta))*(r1**2/R**2)*tan(theta))# 
Lm_wo_leg = (r1**2/R**2)*tan(theta)

In [7]:
subs_dict = {'alpha':mpmath.pi/18, 'k':5,'R':mpf(0.1), 'm':20,'n':10}

In [8]:
Lm = Integral(Lm_expr, (theta,0,alpha))#.doit(meijerg=True)

In [9]:
Lm_func = lambdify([m, R, alpha], Lm, 'mpmath')

In [10]:
%%time 
Lm_func(subs_dict['m'],subs_dict['R'],subs_dict['alpha'])

CPU times: user 21.1 ms, sys: 0 ns, total: 21.1 ms
Wall time: 20.9 ms


mpf('0.000809826285092604393950167190773741')

In [11]:
# eqn 12.107
Kmn_expr = legendre(n, cos(theta))*legendre(m, cos(theta))*sin(theta)
Kmn = Integral(Kmn_expr, (theta, alpha, pi))#.as_sum(num_sumterms,'midpoint')

In [12]:
kmn_lambdify = lambdify([m,n,alpha],Kmn,'mpmath')

In [13]:
%%time 
kmn_lambdify(subs_dict['m'],subs_dict['n'], subs_dict['alpha'])

CPU times: user 169 ms, sys: 0 ns, total: 169 ms
Wall time: 168 ms


mpf('-0.00172754746264863243600335154223454')

In [28]:
Imn_part1 = (n*sph_hankel2(n-1,k*r1)-(n+1)*sph_hankel2(n+1,k*r1))*legendre(n, cos(theta))*cos(theta)
Imn_part2 = n*(n+1)*sph_hankel2(n, k*r1)*(legendre(n-1, cos(theta)-legendre(n+1, cos(theta))))/k*r1
Imn_parts = expand(Imn_part1+Imn_part2)
Imn_expr = expand(sympy.simplify(Imn_parts*legendre(m,cos(theta))*(r1**2/R**2)*tan(theta)))
Imn = Integral(Imn_expr, (theta, 0, alpha))#.doit(meijerg=True)

Imn_lambdify = lambdify([m,n,k,R,alpha], Imn,'sympy')

In [29]:
from sympy.utilities.autowrap import binary_function, autowrap

In [ ]:
%%time
u = Imn_lambdify(subs_dict['m'],
             subs_dict['n'],
             subs_dict['k'],
             subs_dict['R'],
            subs_dict['alpha'])

In [31]:
M_mn = (Imn + (n*sph_hankel2(n-1,k*R) - (n+1)*sph_hankel2(n+1,k*R) )*Kmn)/(2*n+1)



In [42]:
M_mn_func = lambdify((m,n,k,R,alpha), M_mn,'sympy')

In [34]:
b = -I*Lm
b_func = lambdify([m,alpha], b,'mpmath')

In [35]:
frequency = 50*10**3 # kHz
vsound = 330 # m/s
wavelength = vsound/frequency
alpha_value = np.radians(60)
k_value = 2*np.pi/(wavelength)
ka = 5
a_value = ka/k_value 
R_value = a_value/mpmath.sin(alpha_value) # m


In [36]:
Nv = 12 + int(2*ka/sin(alpha_value))
M_matrix = mpmath.matrix(Nv,Nv)
b_matrix = mpmath.matrix(Nv,1)


In [37]:
params = {'k':k_value, 'a':a_value, 'R':R_value, 'alpha':alpha_value}

In [ ]:
M_mn_func

In [40]:
def M_mn_wrapper(eachm,eachn,params):
    return M_mn_func(eachm,eachn,params['k'],params['R'],params['alpha'])

In [43]:
M_mn_wrapper(0,1,params)

KeyboardInterrupt: 

In [39]:
%%time
Mmn(0,1,params)

CPU times: user 105 ms, sys: 70 µs, total: 105 ms
Wall time: 104 ms


KeyboardInterrupt: 

In [24]:
%%time
rowcol = [ ]
for eachn in tqdm.trange(Nv):
    for eachm in range(Nv):
        Imn_value = Imn_lambdify(eachm, eachn, params['k'], params['R'], params['alpha'])
        first_hankel = eachn*sph_hankel2(eachn-1,params['k']*params['R'])
        second_hankel = (eachn+1)*sph_hankel2(eachn+1,params['k']*params['R'])
        Kmn_value = kmn_lambdify(eachm,eachn, params['alpha'])
        M_matrix[eachm, eachm] = ( Imn_value + (first_hankel - second_hankel)*Kmn_value)/(2*eachn+1)


  0%|          | 0/23 [00:00<?, ?it/s]


NameError: name 'jn' is not defined

In [25]:
help(mpmath.lu_solve)

Help on method lu_solve in module mpmath.matrices.linalg:

lu_solve(A, b, **kwargs) method of mpmath.ctx_mp.MPContext instance
    Ax = b => x
    
    Solve a determined or overdetermined linear equations system.
    Fast LU decomposition is used, which is less accurate than QR decomposition
    (especially for overdetermined systems), but it's twice as efficient.
    Use qr_solve if you want more precision or have to solve a very ill-
    conditioned system.
    
    If you specify real=True, it does not check for overdeterminded complex
    systems.



In [26]:
for each_m in range(Nv):
    #M_matrix[each_m, each_n] = M_mn_func(each_m, each_n, k_value, R_value, alpha_value)
    b_matrix[each_m,:] = b_func(each_m, alpha_value)


In [27]:
#a_matrix = mpmath.lu_solve(M_matrix,b_matrix)
a_matrix,res = mpmath.qr_solve(M_matrix,b_matrix)

ValueError: matrix is numerically singular

In [ ]:
mpmath.residual(M_matrix,a_matrix,b_matrix)

In [ ]:
legendre_func = lambdify((m, x), legendre(m, x),'mpmath')

In [ ]:

def d_theta(angle,k_v,R_v,alpha_v,An):
    num = 4 
    N_v = An.rows
    denom  = (k_v**2)*(R_v**2)*mpmath.sin(alpha_v)**2
    part1 = num/denom
    jn_matrix = np.array([1j**f for f in range(N_v)])
    legendre_matrix = np.array([legendre(n_v, np.cos(angle)) for n_v in range(N_v)])
    part2_matrix = np.column_stack((An, jn_matrix, legendre_matrix))
    part2 = np.sum(np.apply_along_axis(lambda X: X[0]*X[1]*X[2], 1, part2_matrix))
    rel_level = - part1*part2
    return rel_level

def d_zero(k_v,R_v,alpha_v,An):
    num = 4 
    N_v = An.rows
    denom  = (k_v**2)*(R_v**2)*mpmath.sin(alpha_v)**2
    part1 = num/denom
    jn_matrix = np.array([1j**f for f in range(N_v)])
    part2_matrix = np.column_stack((An, jn_matrix))
    part2 = np.sum(np.apply_along_axis(lambda X: X[0]*X[1], 1, part2_matrix))
    rel_level = - part1*part2
    return rel_level

def relative_directionality_db(angle,k_v,R_v,alpha_v,An):
    off_axis = d_theta(angle,k_v,R_v,alpha_v,An)
    on_axis = d_zero(k_v,R_v,alpha_v,An)
    rel_level = 20*mpmath.log10(abs(off_axis/on_axis))
    return rel_level

In [ ]:

angles = np.linspace(0,2*np.pi,200)
dirnlty = [relative_directionality_db(angle_v, k_value, R_value,alpha_value,a_matrix) for angle_v in angles]


In [ ]:
plt.figure()
a0 = plt.subplot(111, projection='polar')
plt.plot(angles, dirnlty)
plt.ylim(-40,0);plt.yticks([0,-10,-20,-30,-40]);
plt.xticks(np.radians(np.arange(0,360,30)));